# Household analysis on OpenSAFELY data

Python code for pulling in the full-sized data and running a household regression

The expectation is that this may not converge, but it is intended to check the model pipeline


In [1]:
%matplotlib inline
import numpy as np
import scipy.stats as st
import scipy.optimize as op
import pandas as pd
from numpy import linalg as LA
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns
import logging

In [2]:
logging.basicConfig(filename='./opensafely_age_hh.log', level=logging.INFO, format='%(asctime)s %(message)s')
logging.info('Libraries imported and logging started')

# CHANGE THE NEXT CELL

The next set of cell is the one that would need changing for a different data source in the same format


In [3]:
df = pd.read_stata('../hh_analysis_datasetALLVARS.dta', columns=["hh_id", "age", "case"])

In [4]:
logging.info('Data Read In')

# Data pre-processing

There are two major outputs for each household: a vector y of outcomes and a design matrix X. These are stored in arrays of length equal to the number of households Y and XX respectively.


In [7]:
# This is the number of age classes; here we will follow Roz's interests and consider two young ages

nages = 2
hhnums = pd.unique(df.hh_id)

In [ ]:
# Take a small random sample of households (using a seed for reproducibility) so we can check everything runs
import random
hhnums = random.Random("seed_a4Feji").sample(hhnums.tolist(), min(len(hhnums), 10000))
# 10000 hh's: 1 optimisation run=3 minutes
# 100000 hh's: 1 optimisation run=25 minutes
# 500000 hh's: 1 optimisation run=2 hours
# changed to age categories 100000 hh's: 1 optimisation run=??

# Setting this to 1 as suggested by Roz so we can check how long it takes
optimize_maxiter = 10 #  Previous value was 1000

In [9]:
Y = [] # To store outcomes
XX = [] # To store design matrices
for i, num in enumerate(hhnums):
    dfh = df[df.hh_id == num]    
    
    mya = dfh.age.values
    m = len(mya)
    myx = np.zeros((m,nages))
    myy = np.zeros(m)
    for j, a in enumerate(mya):
        if (a<=9):
            myx[j,:] = np.array([1, 0])
        elif ((a>9) and (a<=18)):
            myx[j,:] = np.array([0, 1])
        if (dfh.case.values[j] == 1):
            myy[j] = 1
    Y.append(myy)
    XX.append(np.atleast_2d(myx))

In [10]:
logging.info('Data pre-processing completed')

# Define functions

We need two functions:

* phi is the Laplace transform of the distribution of heterogeneity in transmissibility
* mynll is the negative log likelihood function for the model

Note that mynll here does not include a 'dilution' effect with number of household occupants (often called the 'Cauchemez model') and there are many other refinements we might like to consider.


In [11]:
def phi(s, logtheta=0.0):
    theta = np.exp(logtheta)
    return ((1.0 + theta*s)**(-1.0/theta))

In [12]:
def mynll(x):
    
    try: # Ideally catch the linear algebra fail directly
        llaL = x[0]
        llaG = x[1]
        logtheta = x[2]
        alpha = x[3:(3+nages)]
        beta = x[(3+nages):(3+2*nages)]
        gamma = x[(3+2*nages):]

        nlv = np.zeros(len(hhnums)) # Vector of negative log likelihoods
        for i in range(0,len(hhnums)):
            y = Y[i]
            X = XX[i]
            if np.all(y==0.0):
                nlv[i] = np.exp(llaG)*np.sum(np.exp(alpha@(X.T)))
            else:
                # Sort to go zeros then ones WLOG (could do in pre-processing)
                ii = np.argsort(y)
                y = y[ii]
                X = X[ii,:]
                q = sum(y>0)
                r = 2**q
                m = len(y)

                # Quantities that don't vary through the sum
                Bk = np.exp(-np.exp(llaG)*np.exp(alpha@(X.T)))
                laM = np.exp(llaL)*np.outer(np.exp(beta@(X.T)),np.exp(gamma@(X.T)))

                BB = np.zeros((r,r)) # To be the Ball matrix
                for jd in range(0,r):
                    for omd in range(0,jd+1):
                        jstr = format(jd,'0' + str(m) + 'b')
                        omstr = format(omd,'0' + str(m) + 'b')
                        j = np.array([int(jstr[x]) for x in range(0,len(jstr))])
                        om = np.array([int(omstr[x]) for x in range(0,len(omstr))])
                        if np.all(om<=j):
                            BB[jd,omd] = 1.0/np.prod((phi((1-j)@laM,logtheta)**om)*(Bk**(1-j)))
                nlv[i] = -np.log(LA.solve(BB,np.ones(r))[-1])
        nll = np.sum(nlv)
        
        nll += 7.4*np.sum(x**2) # Comment out this Ridge if not needed
        
        return nll
    except:
        nll = np.inf
        return nll

In [13]:
logging.info('Helper functions defined')

# Fit the model

The code here uses the simplest kind of maximum likelihood estimation that one might try - it is likely that there may need to be some tuning of this process to the data and computational resources available, and also that in the current context it will fail because almost by definition the model is mis-specified compared to the data.


In [21]:
# Starting parameters - and check that the target function evaluates OK at them

x0 = np.array([
    -2.0,
    -2.0,
    0.0,
    0.0, 0.0,
    0.0, 0.0,
    0.0, 0.0,
])
mynll(x0)

inf

In [22]:
logging.info('Objective function evaluated at one value')

In [23]:
def callbackF(x):
    logging.info('Optimizer callback')
    print('Evaluated at [{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}]: {:.8f}'.format(
        x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],mynll(x)))

In [24]:
bb = np.array([
    [-5.,0.],
    [-5.,0.],
    [-10.,10.],
    [-3.,3.],[-3.,3.],
    [-3.,3.],[-3.,3.],
    [-3.,3.],[-3.,3.],
])

In [25]:
# First try from (essentially) the origin using Nelder-Mead
# The exact optimisation method to use is expected to depend a lot on the actual data
fout = op.minimize(mynll,x0,
                           bounds=bb,method='TNC',callback=callbackF,
                           options={'maxiter' : optimize_maxiter, 'ftol' : 1e-4})
xhat = fout.x
fout

KeyboardInterrupt: 

In [ ]:
pn = len(x0)
delta = 1e-2 # This finite difference needs some unavoidable tuning by hand
dx = delta*xhat
ej = np.zeros(pn)
ek = np.zeros(pn)
Hinv = np.zeros((pn,pn))
for j in tqdm(range(0,pn)):
    ej[j] = dx[j]
    for k in range(0,j):
        ek[k] = dx[k]
        Hinv[j,k] = mynll(xhat+ej+ek) - mynll(xhat+ej-ek) - mynll(xhat-ej+ek) + mynll(xhat-ej-ek)
        ek[k] = 0.
    Hinv[j,j] = - mynll(xhat+2*ej) + 16*mynll(xhat+ej) - 30*mynll(xhat) + 16*mynll(xhat-ej) - mynll(xhat-2*ej)
    ej[j] = 0.
Hinv += np.triu(Hinv.T,1)
Hinv /= (4.*np.outer(dx,dx) + np.diag(8.*dx**2)) # TO DO: replace with a chol ...
covmat = LA.inv(0.5*(Hinv+Hinv.T))
stds = np.sqrt(np.diag(covmat))
stds

In [ ]:
logging.info('One optimisation run')

In [24]:
print('Baseline probability of infection from outside is {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*(1.-np.exp(-np.exp(xhat[1]))),
    100.*(1.-np.exp(-np.exp(xhat[1]-1.96*stds[1]))),
    100.*(1.-np.exp(-np.exp(xhat[1]+1.96*stds[1]))),
    ))

mymu = xhat[[0,2]]
mySig = covmat[[0,2],:][:,[0,2]]
m = 4000
sarvec = np.zeros(m)
try:
    for i in range(0,m):
        uu = np.random.multivariate_normal(mymu,mySig)
        sarvec[i] = 100.*(1.-phi(np.exp(uu[0]),uu[1]))
except ValueError as e:
    if str(e) == "array must not contain infs or NaNs":
        print("Unable to compute baseline SAR, got: {!r}".format(e))
    else:
        raise
else:
    print('Baseline SAR is {:.1f} ({:.1f},{:.1f}) %'.format(
        100.*(1.-phi(np.exp(xhat[0]),xhat[2])),
        np.percentile(sarvec,2.5),
        np.percentile(sarvec,97.5),
        ))

print('Relative external exposure for <=9yo {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*np.exp(xhat[3]),
    100.*np.exp(xhat[3]-1.96*stds[3]),
    100.*np.exp(xhat[3]+1.96*stds[3]),
    ))
print('Relative external exposure for 10-18yo {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*np.exp(xhat[4]),
    100.*np.exp(xhat[4]-1.96*stds[4]),
    100.*np.exp(xhat[4]+1.96*stds[4]),
    ))

print('Relative susceptibility for <=9yo {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*np.exp(xhat[5]),
    100.*np.exp(xhat[5]-1.96*stds[5]),
    100.*np.exp(xhat[5]+1.96*stds[5]),
    ))
print('Relative susceptibility for 10-18yo {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*np.exp(xhat[6]),
    100.*np.exp(xhat[6]-1.96*stds[6]),
    100.*np.exp(xhat[6]+1.96*stds[6]),
    ))

print('Relative transmissibility for <=9yo {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*np.exp(xhat[7]),
    100.*np.exp(xhat[7]-1.96*stds[7]),
    100.*np.exp(xhat[7]+1.96*stds[7]),
    ))
print('Relative transmissibility for 10-18yo {:.1f} ({:.1f},{:.1f}) %'.format(
    100.*np.exp(xhat[8]),
    100.*np.exp(xhat[8]-1.96*stds[8]),
    100.*np.exp(xhat[8]+1.96*stds[8]),
    ))


Baseline probability of infection from outside is 26.2 (24.2,28.4) %


/Users/thomas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  
/Users/thomas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/thomas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


Baseline SAR is 0.0 (nan,nan) %
Relative external exposure for <=9yo 103.1 (79.5,133.8) %
Relative external exposure for 10-18yo 120.0 (93.3,154.5) %
Relative susceptibility for <=9yo 6473745680521278870101517409490556427432005355849479658811821610514461282919161647515900567355392.0 (0.0,inf) %
Relative susceptibility for 10-18yo 0.0 (nan,nan) %
Relative transmissibility for <=9yo 523799530438513999240421668946643127191119637512192.0 (nan,nan) %
Relative transmissibility for 10-18yo 1155675035405068003189539162429240125574523855850082140160.0 (nan,nan) %


/Users/thomas/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [ ]:
logging.info('Final notebook cell evaluated')